In [ ]:
import numpy as np
import pandas as pd
import tqdm

import traceback
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from os_elm import OS_ELM


In [ ]:
def softmax(label_classify):
    c = np.max(label_classify, axis=-1).reshape(-1, 1)
    exp_label_classify = np.exp(label_classify- c)
    sum_exp_label_classify = np.sum(exp_label_classify, axis=-1).reshape(-1, 1)
    return exp_label_classify/sum_exp_label_classify 

# Instantiate os-elm
# ===========================================
def initialiseOSELM(n_input_nodes, n_hidden_nodes, n_output_nodes):
    print("initialiseOSELM ===========================================")
    try:
        os_elm = OS_ELM(
            # the number of input nodes.
            n_input_nodes=n_input_nodes,
            # the number of hidden nodes.
            n_hidden_nodes=n_hidden_nodes,
            # the number of output nodes.
            n_output_nodes=n_output_nodes,
                 
            # NOTE: OS-ELM apply an activation function only to the hidden nodes.
            activation='sigmoid',
        )
    except Exception as e:
        traceback.print_exc()
        raise e
    return os_elm

In [ ]:
def prapareData(x_classify, t_classify, n_classes, n_hidden_nodes):
   
    # dataset preparation
    # ===========================================
    print("prapareData ===========================================")
    print(f'x_classify sahpe : {x_classify.shape}')
    print(f't_classify sahpe : {t_classify.shape}')

    # shuffle dataset
    perm = np.random.permutation(len(x_classify))
    x_classify = x_classify[perm]
    t_classify = t_classify[perm]

    # divide dataset for training and testing
    split_border = int(len(x_classify) * 0.98)

    x_train, x_test = x_classify[:split_border], x_classify[split_border:]
    t_train, t_test = t_classify[:split_border], t_classify[split_border:]
    print(f'Training data from 0 - {split_border}')
    print(f'Test data from {split_border} - {len(t_classify)}')

    min_max_scalar = MinMaxScaler().fit(x_train)
    x_train_normalised = min_max_scalar.transform(x_train)
    x_test_normalised = min_max_scalar.transform(x_test)
    print(f'x_train_normalised sahpe : {x_train_normalised.shape}')
    print(f'x_test_normalised sahpe : {x_test_normalised.shape}')

    t_test_categorised = to_categorical(t_test, num_classes=n_classes)
    t_train_categorised = to_categorical(t_train, num_classes=n_classes)
    print(f't_test_categorised sahpe : {t_test_categorised.shape}')
    print(f't_train_categorised sahpe : {t_train_categorised.shape}')

    # divide dataset for initial training
    initial_train_border = int(n_hidden_nodes)
    print(f'Initial Training data border from - {initial_train_border}')
    x_trainclassify_init = x_train_normalised[:initial_train_border]
    x_train_seq = x_train_normalised[initial_train_border:]
    print(f'x_trainclassify_init sahpe : {x_trainclassify_init.shape}')
    print(f'x_train_seq sahpe : {x_train_seq.shape}')

    t_trainclassify_init = t_train_categorised[:initial_train_border]
    t_train_seq = t_train_categorised[initial_train_border:]
    print(f't_trainclassify_init sahpe : {t_trainclassify_init.shape}')
    print(f't_train_seq sahpe : {t_train_seq.shape}')
    return x_train_normalised, t_train_categorised, x_trainclassify_init, t_trainclassify_init, x_train_seq, t_train_seq, x_test_normalised, t_test_categorised

In [ ]:
def trainModel(os_elm, x_train_normalised, x_trainclassify_init, t_trainclassify_init, x_train_seq, t_train_seq):
    print("trainModel ===========================================")
    pbar = tqdm.tqdm(total=len(x_train_normalised), desc='initial training phase')
    try:
        # ===========================================
        # Training
        # ===========================================
        # the initial training phase
        try:
            print(f'******************inital training Start************')
            os_elm.init_train(x_trainclassify_init, t_trainclassify_init)
            print(f'******************inital training Complete************')
        except Exception as e:
            traceback.print_exc()
        finally:
            pbar.update(n=len(x_trainclassify_init))
            # the sequential training phase
            
            # ===========================================
            # Training
            # ===========================================
            # the sequential training phase
        pbar.set_description('Sequential training phase')
        batch_size = 11
        print(f'******************Sequential training Start************')
        print(f'len(x_train_seq) :: {len(x_train_seq)}')
        for i in range(0, len(x_train_seq), batch_size):
            x_batch = x_train_seq[i:i+batch_size]
            t_batch = t_train_seq[i:i+batch_size]
            os_elm.seq_train(x_batch, t_batch)
            pbar.update(n=len(x_batch))    
    except Exception as e:
        traceback.print_exc()
        raise e
    finally:
        pbar.close()

In [ ]:
def predictOnTestSample(os_elm, n, x_test_normalised, t_test_categorised):
    print("predictOnTestSample ===========================================")
    # ===========================================
    # Prediction
    # ===========================================
    # sample 10 validation samples from x_test
    # n = 10
    x = None
    t = None
    y = None
    x = x_test_normalised[:n]
    t = t_test_categorised[:n]

    # 'predict' method returns raw values of output nodes.

    try:
        y = os_elm.predict(x)
        # print(f'Y result{y}')
    except Exception as e:
        traceback.print_exc()
        raise e
    # apply softmax function to the output values.
    y = softmax(y)
    # print(f'SOFTmax result{y}')

    # check the answers.
    max_ind_classify = None
    for i in range(len(y-1)):
    # for i in range(n):
        max_ind_classify = np.argmax(y[i])
        print('========== index number %d ==========' % i)
        print('estimated class %d' % max_ind_classify)
        print('estimated probability: %.3f' % y[i,max_ind_classify])
        print('true answer: class %d' % np.argmax(t[i]))

In [ ]:
def evaluateTestData(os_elm, x_test_normalised, t_test_categorised, metrics=['ttime', 'accuracy']):
    # ===========================================
    # Evaluation

    try:
        [ttime, accuracy] = os_elm.evaluate(x_test_normalised, t_test_categorised, metrics=metrics)
        print(f'time: {ttime} , accuracy: {accuracy}')
    except Exception as e:
        traceback.print_exc()
        raise e

def saveModel(os_elm):
    # ===========================================
    # Save model
    # ===========================================
    print('saving model parameters...')
    os_elm.save('./checkpoint/model.ckpt')
    print('model parameters saved')

    # initialize weights of os_elm
    os_elm.initialize_variables()


In [ ]:
# ****************************************
# Initialise Variables
# ****************************************
initialise_os_elm = True
train_model = True
load_data = True
n_input_nodes = 3
n_hidden_nodes = 5
n_output_nodes = 5
n_classes = n_output_nodes
start_data_limit = 0
end_data_limit = 50000

t_classify = None
x_classify = None
x_classify_df = None
os_elm = None
input_data = None
pd.options.mode.chained_assignment = None
input_Data_length = 0
count = 1
iteration_limit = 100
n = 0
while train_model:
    print(f"start_data_limit : {start_data_limit}, end_data_limit : {end_data_limit}")
    # # ===========================================
    # # Load dataset
    # # ===========================================
    if initialise_os_elm:
        os_elm = initialiseOSELM(n_input_nodes, n_hidden_nodes, n_output_nodes)
        initialise_os_elm = False

    # ===========================================
    # Load dataset
    # ===========================================
    if load_data:
        input_data = pd.read_csv("TrainingData20K.csv")
        input_Data_length = len(input_data.index)
        print(f'Total records Loaded : {len(input_data.index)}')
        load_data = False
        
        x_classify_df = input_data[['Tot_Fwd_Pkts','Fwd_Header_Len','Fwd_IAT_Max', 'Label']]

        x_classify_df_with_label = x_classify_df
   
    x_classify_df = x_classify_df_with_label[['Tot_Fwd_Pkts','Fwd_Header_Len','Fwd_IAT_Max']]
    x_classify=np.array(x_classify_df[start_data_limit:end_data_limit])
    t_classify = x_classify_df_with_label['Label']
    t_classify=np.array(t_classify[:len(x_classify)])

    start_data_limit = end_data_limit
    n = n+2;
    end_data_limit = end_data_limit + (n + n_hidden_nodes)
    
    # ===========================================
    # Prepare dataset
    # ===========================================
    x_train_normalised, t_train_categorised, x_trainclassify_init, t_trainclassify_init, x_train_seq, t_train_seq, x_test_normalised, t_test_categorised = prapareData(x_classify, t_classify, n_classes, n_hidden_nodes)
    # ===========================================
    # Training Model
    # ===========================================
    trainModel(os_elm, x_train_normalised, x_trainclassify_init, t_trainclassify_init, x_train_seq, t_train_seq)
    # ===========================================
    # Prediction Sample 10 records
    # ===========================================
    # predictOnTestSample(os_elm, 10, x_test_normalised, t_test_categorised)
    # ===========================================
    # Evaluation
    # ===========================================
    evaluateTestData(os_elm, x_test_normalised, t_test_categorised, metrics=['ttime', 'accuracy'])
    # ===========================================
    # Save model
    # ===========================================
    saveModel(os_elm)

    count = count +1
    if end_data_limit > input_Data_length or count > iteration_limit:
        print(f"end_data_limit : {end_data_limit}, input_Data_length : {input_Data_length}")
        train_model = False

    